In [1]:
%autosave 0

<IPython.core.display.Javascript object>

Autosave disabled


# [Rubik's cube 3x3x3](https://fr.wikipedia.org/wiki/Rubik%27s_Cube#:~:text=Le%20Rubik's%20Cube%20est%20un,ar%C3%AAte%20%C3%A0%20deux%20faces%20visibles.) 

## Avertissement
Le présent Notebook n'est qu'une interface aux méthodes,  déjà implémentées dans Sage (avec le package `rubiks`), de résolution d'un Rubik's Cube. Les outils Sage relatifs au Rubik's Cube sont principalement consacrés à la structure de groupe du cube et, même si Sage permet de résoudre une configuration donnée, il faut lui fournir cette configuration sous la forme de ce que Sage appelle l'*état* ou `state` associé à cette configuration ; voici par exemple l'état du cube résolu :

```python
solvedState = {'right': [[25, 26, 27], [28, 0, 29], [30, 31, 32]],
               'left': [[9, 10, 11], [12, 0, 13], [14, 15, 16]],
               'up': [[1, 2, 3], [4, 0, 5], [6, 7, 8]],
               'down': [[41, 42, 43], [44, 0, 45], [46, 47, 48]],
               'front': [[17, 18, 19], [20, 0, 21], [22, 23, 24]],
               'back': [[33, 34, 35], [36, 0, 37], [38, 39, 40]]}
```

L'essentiel de ce qui suit consiste donc essentiellement à calculer l'état d'une configuration à partir de sa représentation donnée par la suite des couleurs des petits cubes la constituant, ce qui n'est pas complètement évident (ni trés intéressant d'ailleurs $\ldots$).

## Objet du problème

On se propose d'écrire une fonction qui prend en entrée une position du cube et calcule une solution.

Chacune des 6 couleurs est représentée par l'initiale de son nom en anglais : `o`range, `b`lue, `r`ed, `w`hite, `y`ellow, `g`reen.

Une position est représentée par une chaine `s` de 54 caractères selon le schéma suivant qui représente le cube développé :

Le cube est orienté de sorte que l'on ait `s[4]` $=$ `'y'`, `s[13]` $=$ `'b'`, `s[22]` $=$ `'r'`, `s[31]` $=$ `'g'`, `s[40]` $=$ `'o'` et `s[49]` $=$ `'w'`.

Le cube résolu par exemple est représenté par :
```python
solvedS = 'yyyyyyyyybbbbbbbbbrrrrrrrrrgggggggggooooooooowwwwwwwww'
```

Une solution sera donnée sous la forme d'une chaine de caractères avec les conventions données [là](https://ruwix.com/the-rubiks-cube/notation/)
où, ici, la face avant, __F__, est la face (dont la case centrale est) rouge, la face gauche, __L__, est la face ($\ldots$) bleue, __U__ est la jaune, etc.

Par exemple, pour
```python
s = 'gorbygorbryyobrrwwgyygrbgwwoyyrgoowwbyybogbwworbbwggor'
```
une solution est
```python
"D R2 L' B' R' U2 R2 U F D U2 L D2 F' D2 R2 F' D2 B2 U2 R2 U2 L2 F2"
```
à lire de gauche à droite.


## Implémentation

### Fonction essentielle

```python
state_of_string(s)
```
et une fonction utilitaire :
```python
inverse(sol)
```

In [2]:
import numpy as np

def initRubikCube():

    global colors, color, cubes, cubesOfColors
    
    colors = [(1,1,0), (0,0, 1), (1, 0, 0), (0, 1, 0), (1,.5,0), (1,1, 1), (0.1, 0.1, 0.1)]

    x = np.zeros(49, dtype=np.uint8)
    y = np.zeros(49, dtype=np.uint8)
    z = np.zeros(49, dtype=np.uint8)
    for i in [1,2,3,9,12,14,27,29,32,33,34,35,36,37,38,39,40,46,47,48]: x[i] = 0
    for i in [4,5,10,15,26,31,44,45]: x[i] = 1
    for i in [6,7,8,11,13,16,17,18,19,20,21,22,23,24,25,28,30,41,42,43]: x[i] = 2
    for i in [1,4,6,9,10,11,12,13,14,15,16,17,20,22,35,37,40,41,44,46]: y[i] = 0
    for i in [2,7,18,23,34,39,42,47]: y[i] = 1
    for i in [3,5,8,19,21,24,25,26,27,28,29,30,31,32,33,36,38,43,45,48]: y[i] = 2
    for i in [14,15,16,22,23,24,30,31,32,38,39,40,41,42,43,44,45,46,47,48]: z[i] = 0
    for i in [12,13,20,21,28,29,36,37]: z[i] = 1
    for i in [1,2,3,4,5,6,7,8,9,10,11,17,18,19,25,26,27,33,34,35]: z[i] = 2
        
    color = np.zeros(49, dtype='<U1')
    for i in range(1,9): color[i] = 'y'
    for i in range(9,17): color[i] = 'b'
    for i in range(17,25): color[i] = 'r'
    for i in range(25,33): color[i] = 'g'
    for i in range(33,41): color[i] = 'o'
    for i in range(41,49): color[i] = 'w'
        
    cubes = dict()
    for i in range(3):
        for j in range(3):
            for k in range(3):
                cubes[(i,j,k)] = set()
    for n in range(1,49):
        cubes[(x[n], y[n], z[n])].add(n)
            
    cubes = [cubes[t] for t in cubes.keys()]
    
    cubesOfColors = dict()
    for c in cubes:
        s = frozenset([color[i] for i in c])
        cubesOfColors[s] = c

initRubikCube()

def inverse(s):
    
    inv = {"U" : "U'", "L" : "L'", "F" : "F'", "R" : "R'", "B" : "B'", "D" : "D'",
           "U'" : "U", "L'" : "L", "F'" : "F", "R'" : "R", "B'" : "B", "D'" : "D"}
    
    l = [inv[x] if x in inv.keys() else x for x in s.split(' ')]
    l.reverse()
    return ' '.join(l) 
    
def state_of_string(s):
    
    if s[4] != 'y' or s[13] != 'b'or s[22] != 'r' or s[31] != 'g'or s[40] != 'o' or s[49] != 'w': 
        raise Exception('configuration illégale')
    s0 = '#' + s[:4] + s[5:13] + s[14:22] + s[23:31] + s[32:40] + s[41:49] + s[50:]
    
    d = np.zeros(49, dtype = np.uint8)
    for k, c in enumerate(s0):
        if k:
            cols = frozenset({s0[h] for h in [c for c in cubes if k in c][0]})
            cube =  cubesOfColors[cols]
            for h in cube:
                if color[h] == c:
                    d[h] = k
    
    stateInit = CubeGroup().faces('')
    state = dict()
    for key, val in stateInit.items():
        state[key] = [list(map(lambda k: d[k], l)) for l in stateInit[key]]
    
    if not CubeGroup().legal(state):
        raise Exception('configuration illégale')
    
    return state

### Dessins de flèches courbes
pour la représentation graphique d'une solution

In [3]:
alpha, beta = .7, .5
u0, v0, w0 = np.array((1.,0.,0.)), np.array((0.,1.,0.)), np.array((0.,0.,1.))
u, v, w, x, y, z = alpha * u0, alpha * v0, alpha * w0, beta * u0, beta * v0, beta * w0
fleches = {
    "U" : (z, v, u, '1'),
    "D" : (-z, u, v, '1'),
    "R" : (y, u, w, '1'),
    "L" : (-y, w, u, '1'),
    "F" : (x, w, v, '1'),
    "B" : (-x, v, w, '1'), 
    "U'" : (z, u, v, '1'),
    "D'" : (-z, v, u, '1'),
    "R'" : (y, w, u, '1'),
    "L'" : (-y, u, w, '1'),
    "F'" : (x, v, w, '1'),
    "B'" : (-x, w, v, '1'),
    "U2" : (z, v, u, '2'),
    "D2" : (-z, u, v, '2'),
    "R2" : (y, u, w, '2'),
    "L2" : (-y, w, u, '2'),
    "F2" : (x, w, v, '2'),
    "B2" : (-x, v, w, '2')
}

def fleche(x, v, radius, **kwargs):

    from sage.plot.plot3d.shapes import Cone

    x = np.array(x)
    v = np.array(v)
    height = (sum(v**2))**.5
    k = height * np.array([0,0,1])
    return Cone(radius, height, closed = False, **kwargs).rotate(k + v, pi).translate(x)


def arc(centre = (0.,0.,0.), u = (1.,0.,0.), v = (0.,1.,0.),
        text = None, I = (.3,pi/2. - .3), arrow = True, **kwargs):
    
    centre = np.array(centre)
    u = np.array(u)
    v = np.array(v)
    t = var('t')
    G = parametric_plot3d(list(centre + cos(t) * u + sin(t) * v), (t, *I), linewidth = 5, **kwargs)
    t1,t2 = I
    if arrow:
        G += fleche(centre + cos(t2) * u + sin(t2) * v, .15 * (-sin(t2) * u + cos(t2) * v), .02, **kwargs)
    if text is not None:
        t3 = t1 - .1
        G += text3d(text, centre + cos(t3) * u + sin(t3) * v, fontsize = 30, **kwargs)
    return G

arcs = dict()
for x, v in fleches.items():
    arcs[x] = arc(*v, color = 'goldenrod').plot()

### Fonction principale
```python
solve_rubik3(s, algorithm='hybrid', timeout=15)
```
`algorithm` doit avoir une des valeurs suivantes :

- 'hybrid' - essaie 'kociemba' pendant `timeout` secondes, puis 'dietz'

- 'kociemba' - Utilise le programme de Dik T. Winter (vitesse raisonnable, peu de mouvements)

- 'dietz' - Utilise le programme cubex de Eric Dietz (rapide mais beaucoup de mouvements)

- 'optimal' - Utilise le programme optimal de Michael Reid (peut durer longtemps)

Affiche une solution `sol` et le nombre $n$ de mouvements et
renvoie le couple `(sol, G)` où  `G` est un itérateur qui fournit une représentation graphique de chacun des mouvements nécessaires à la résolution : executer `next(G)` $n$ fois.

On dispose aussi de la fonction 
```python
verification(s, sol)
```
qui doit renvoyer `True`

In [4]:
def solve_rubik3(s, **kwargs):
    
    C = RubiksCube(colors = colors, state = state_of_string(s))
    sol = inverse(C.solve(**kwargs))
    solSplit = sol.split(' ')
    print(f'{sol} ({len(solSplit)})')
    
    def G():
        nonlocal C
        for x in solSplit:
            yield (arcs[x] + C).show(frame = False)
            C = C.move(x)
        yield C.plot3d().show(frame = False)
        
    return sol, G()
    
def verification(s, sol):
    return CubeGroup().faces(inverse(sol)) == state_of_string(s)

## Test

In [12]:
s = 'gorbygorbryyobrrwwgyygrbgwwoyyrgoowwbyybogbwworbbwggor'
sol, G = solve_rubik3(s, algorithm = 'kociemba')
verification(s, sol)

D R2 L' B' R' U2 R2 U F D U2 L D2 F' D2 R2 F' D2 B2 U2 R2 U2 L2 F2 (24)


True

In [39]:
# executer plusieurs fois après avoir executé la cellule précédente
next(G)